<b>Import necessery libraries</b>


In [32]:
#!pip install openpyxl

In [33]:
import spacy
import string
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from azureml.core import Experiment
from azureml.core import Workspace, Dataset
from azureml.data import DataType
from spacy.cli.download import download as spacy_download
import os 
from os.path import join as osjoin
import xlwt
import openpyxl

In [34]:
#!pip install --upgrade spacy

<b>Select the default workspace & datastore</b>

In [35]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '6ed9d167-b2e6-41b8-9500-35e6df64d9dc'
resource_group = 'MLRG'
workspace_name = 'erbbimlws'

workspace = Workspace(subscription_id, resource_group, workspace_name)


datastore = workspace.get_default_datastore()



<b>Loading the Greek language tools</b>

In [36]:
spacy_download('el_core_news_sm')
nlp =spacy.load('el_core_news_sm', disable=['tagger', 'parser', 'ner'])

✔ Download and installation successful
You can now load the package via spacy.load('el_core_news_sm')


<b>Parameter definitions</b>

In [37]:
#minimum number of tokens in the texts
minCount = 1000
#ngrams parameters
mindf,minngram,maxngram = 30,2,3
#keep empty tokens
deleteEmptyTokens = True
#dataset name to be analyzed
datasetName = 'Txns_NLP_202102'
#export filename
fileName = 'Txns_NLP_202102_exp'

<b>Regular expressions definitions</b>

In [38]:
p1 = re.compile('δεν απαντ.{1,3}\s{0,1}',re.IGNORECASE)
p2 = re.compile('\sδα\s',re.IGNORECASE)
p3 = re.compile('δε.{0,1}\s.{0,3}\s{0,1}βρ.{1,2}κ.\s{0,1}',re.IGNORECASE)
p4 = re.compile('[^\d]?\d{10}')
p5 = re.compile('[^\d]?\d{18}|[^\d]\d{20}')
p6 = re.compile('δε[ ν]{0,1} (επιθυμ[α-ω]{2,4}?|[ήη]θ[εέ]λ[α-ω]{1,3}?|θελ[α-ω]{1,4}|.{0,20}ενδιαφ[εέ]ρ[α-ω]{2,4})',re.IGNORECASE)
p7 = re.compile('δε[ ν]{0,1} (μπορ[α-ω]{2,5}|.εχει)',re.IGNORECASE)
p8 = re.compile('(δεν|μη).*διαθεσιμ[οη]ς{0,1}?',re.IGNORECASE)
p9 = re.compile('(δεν|μη)+.*εφικτη?',re.IGNORECASE)
p10 = re.compile('δε[ ν]{0,1}.{0,20}θετικ[οόήη]ς{0,1}',re.IGNORECASE)

#pinakides
#p11 = re.compile('\s([a-zA-Zα-ωΑ-Ω]{3}\s*?[0-9]{3,4})\s',re.IGNORECASE)
p11 = re.compile('\s[a-zA-Zα-ωΑ-Ω]{3}[0-9]{3,4}\s',re.IGNORECASE)

#enoikia
p12=re.compile('εν(.{1,2}κ.{1,3})',re.IGNORECASE)
p13=re.compile('en(.{1,2}k.{1,3})',re.IGNORECASE)

p14 = re.compile('(γιατρος|giatros|φυσιοθεραπ|ορθοπαιδικος|ΩΡΛ|ΩΝΑΣΕΙΟ|ψυχολογος|ψυχοθεραπεια|ψυχιατρος|LOGO -ERGOTHERAPEIA|VIODIAGNOSI|KLINIKI|ΚΛΙΝΙΚΗ)\S*' ,re.IGNORECASE)
p15 = re.compile('(EX/SH|Εξοφληση|ΕΞΟΦΛΗΣΗ|eksoflisi|eksoflhsh|ΕΞΩΦΛΗΣΗ|Εξόφλησ|EXOFLISI|EXOFLHSH|EJOFLHSH|EΞΟΦΛ|εξοφ|εξοφλ|εξοφλη|εξοφλης|εξόφληση|εξφ|εξωφ|εξωφλ)\S*' ,re.IGNORECASE)
p16 = re.compile('(ASFALIA|asfaleia|ασφάλ|asfalistra|ΑΣΦΑΛΕΙΑ|ΑΣΦΑΛΕΙΑΣ|ασφαλειες|ασφάλειες|ασφαλιστηριο|ασφαλιστρα|ασφαλιστρων)\S*' ,re.IGNORECASE)
p17 = re.compile('(ΤΙΜΟΛΟΓΙΟ|ΤΙΜΟΛ|ΤΙΜΟΛΟΓΊΟΥ|ΠΡΟΦΟΡΜΑ|PROFORMA|INVOICE|TIMOLOGIWN)\S*' ,re.IGNORECASE)
p18 = re.compile('(ΝΕΡΟΥ|ΝΕΡΟ|ευδαπ|nero|nerou|eydap|PLIROMI NEROU|ΠΛΗΡΩΜΗ ΝΕΡΟΥ)\S*' ,re.IGNORECASE)
p38 = re.compile('(^|\s)(ΔΕΗ|dei|DEH|ρευμα|reuma|ILEKTRIKOU|ILEKTRIKO|ηλεκτρικο)\S*' ,re.IGNORECASE)
p19 = re.compile('(ΜΙΣΘΟΔΟΣΙΑ|MISTHODOSIA|DORO CHRISTOUGENNON|DORO PASHA|DORO PASXA|apodoches|apodochon|μισθ|μισθου|salary|MISTH|Αδεια|ΑΠΟΔΟΧΕΣ|αποδοχων|μηνιατικο|μισθοδοσία|ΕΞΟΦΛ ΜΙΣΘ|ΜΙΣΘΟΣ|MISTHOS|misthou|ΜΙΣΘΟΔΩΣΙΑ|ΜΙΣΘ/ΣΙΑ|ΔΩΡΟ ΧΡΙΣΟΥΓΕΝΩΝ|ΔΩΡΟ ΧΡΙΣΤ|DWRO XRISTOYGENNWN|ΔΩΡΟ ΠΑΣΧΑ|ΟΔΟΙΠΟΡΙΚΑ|PAYROLL|MISTHODOSIAS|MISTHODOSIES|ODOIPORIKA|EPIDOMA ADEIAS|ΕΠΙΔΟΜΑ ΑΔΕΙΑΣ)\S*' ,re.IGNORECASE)
p41 = re.compile('(ΕΝΦΙΑ|enfia)\S*' ,re.IGNORECASE)
p20 = re.compile('(ΤΑΠ ΑΚΙΝΗΤΟΥ|ΚΤΗΜΑΤΟΛΟΓΙΟ)\S*' ,re.IGNORECASE)
p21 = re.compile('(^|\s)(foros|φορος|φορος εισοδηματος|FOROI|eforia|εφορια)\S*' ,re.IGNORECASE)
p42 = re.compile('(^|\s)(ΕΦΚΑ|efka)(\s|$)' ,re.IGNORECASE)
p43 = re.compile('(fpa|ΦΠΑ)\S*' ,re.IGNORECASE)
p44 = re.compile('(^|\s)(ΙΚΑ|IKA)(\s|$)',re.IGNORECASE)
p45 = re.compile('(^|\s)(TEBE|TEVE|ΤΕΒΕ|oaee|ΟΑΕΕ)(\s|$)' ,re.IGNORECASE)
p22 = re.compile('(KOINOXΡHΣTA|ΚΟΙΝ/ΣΤΑ|κοινοχρηστων|κοινοχρ|Συντηρηση Ανελκυστηρα|Κοινόχροιστα|KOINOXRISTA|KOINOCHRISTA|PLIROMI KOINOCHRISTON|koinoxrhsta)\S*' ,re.IGNORECASE)
p23 = re.compile('(να ζησετε|δωρο γαμου|na zisete|gamou|gamos|γαμος)\S*' ,re.IGNORECASE)
p24 = re.compile('(να σας ζησει|na sas zisei|βαφτιση|vaftisi|baftisi)\S*' ,re.IGNORECASE)
p25 = re.compile('(stegastiko|δοση σπίτι|Δάνειο σπίτι|δόση στεγαστικού δανείου|στεγαστικου|στεγαστικο|ΣΤΕΓΑΣΤΙΚΟ  ΔΑΝΕΙΟ|Πληρωμη στεγαστικου|mortgage installment|mortgage|STEGASTIKOU|EPISKEVASTIKOU|επισκευαστικο|episkeuastiko)\S*' ,re.IGNORECASE)
p26 = re.compile('(μτφ|metafora|μεταφορα|mtf|METAF)\S*' ,re.IGNORECASE)
p27 = re.compile('(ORDER|παραγγελίας|ΠΑΡΑΓΓΕΛΙΑ|αρ.παρ.|PARAGGELIAS|PARAGGELIA|αρ.παρ|orders|PARANGELIA)\S*' ,re.IGNORECASE)
p28 = re.compile('(xreos|ΔΑΝΕΙΚΑ|ΕΝΑΝΤΙ ΛΟΓΑΡΙΑΣΜΟΥ|εξοδα|ΕΞΟΔ.|exoda|expenses|οφειλη|χρεος|ENANTI LOGARIASMOU|DANEIKA|OFEILES)\S*' ,re.IGNORECASE)
p29 = re.compile('(κτθ|καταθεση|katathesi|ktth)\S*' ,re.IGNORECASE)
p30 = re.compile('(ΠΡΟΚΑΤΑ|ΠΡΟΚΑΤΑΒΟΛΉ|ΠΡΟΚΑΤΑΒ|prokatavoli|prokataboli|prokatav|PROKATABOLH|προκατ|προκαταβολες)\S*' ,re.IGNORECASE)
p31 = re.compile('(DIATROFI|ΔΙΑΤΡΟΦΗ|ΔΙΑΤΡΟΦΙ|DIA/FI|ΔΙΑ/ΦΗ)\S*' ,re.IGNORECASE)
p32 = re.compile('(δοση αυτοκινητου|δανειο αυτοκινητου|δανειο αυτοκινητο|Πληρωμη Αυτοκινητο|Αυτοκινητο πληρωμη|αυτοκινητο δοση|daneiou autokinitou|daneio autokinitou|dosi autokonitou|daneiou autokinito)\S*' ,re.IGNORECASE)
#p76 = re.compile('^(αυτοκινητο|autokinito)$' ,re.IGNORECASE)
p76 = re.compile('(^|\s)(αυτοκινητο|autokinito|αυτοκινητου)\S*' ,re.IGNORECASE)
p77 = re.compile('(agora autokinitou|αγορα αυτοκινητου)\S*' ,re.IGNORECASE)
p33 = re.compile('(ΠΛΗΡΩΜΗ ΔΑΝΕΙΟΥ|DANEIOU|δανειο|δανειου|daneioy|δαν|δανε|dosi daniou|εξοικονομω|Πληρωμη δοσης|δοση δανειου|ΔΟΣΗ ΔΑΝΕΙΟΥ|DOSΗ ΔΑΝΕΙΟΥ|ΔΌΣΗΔΑΝΕΊΟΥ|ΠΛΥΡΟΜΗ.ΔΑΝΕΙΟΥ|PLIROMI DANEIOU|DOSI DANEIOU|PLHPOMH DANEIOY|PLIROMI DOSIS DANEIOU|danio|DANEIO )\S*' ,re.IGNORECASE)
p34 = re.compile('(Πληρωμή καρτας|ΠΛΗΡΩΜΉ ΠΙΣΤΩΤΙΚΗΣ|card pay|card payment)\S*' ,re.IGNORECASE)
p47 = re.compile('(dwrea|donation|δωρεα|dorea)\S*' ,re.IGNORECASE)
p48 = re.compile('(dosi|dosis|doseis|dosh|δοση|δοσης|δοσεις)\S*' ,re.IGNORECASE)
p49 = re.compile('(ALPHA BANK|PEIRAIOS|ETHNIKI|PIRAEUS|ALPHA)\S*' ,re.IGNORECASE)
p70 = re.compile('(^|\s)(ετε|ete)(\s|$)',re.IGNORECASE)
p50 = re.compile('(ELLINOAMERIKANIKI|EKPAIDEFTIRIA|EKPAIDEFTIKA|ELLINOAMERIKANIKO|ΕΛΛΗΝΟΑΜΕΡΙΚΑΝΙΚΗ|ΕΛΛΗΝΟΑΜΕΡΙΚΑΝΙΚΟ|ΕΚΠΑΙΔΕΥΤΗΡΙΑ)\S*' ,re.IGNORECASE)
p51 = re.compile('(ΟΑΕΔ|OAED)\S*' ,re.IGNORECASE)
p52 = re.compile('(ΤΕΛΗ ΚΥΚΛΟΦΟΡΙΑΣ|TELI KYKLOFORIAS)\S*' ,re.IGNORECASE)
p53 = re.compile('(ΕΠΙΤΑΓΗ|ΕΠΙΤΑΓΕΣ|ΕΠΙΤΑΓΗΣ|EPITAGHS|EPITAGES|EPITAGH|EPITAGI)\S*' ,re.IGNORECASE)
p54 = re.compile('(MERISMATOS|MERISMA|MERISMATA|MERISMATON|ΜΕΡΙΣΜΑ|ΜΕΡΙΣΜΑΤΑ)\S*' ,re.IGNORECASE)
p55 = re.compile('(COLLEGE|ΚΟΛΛΕΓΙΟ|kolegio)\S*' ,re.IGNORECASE)
p56 = re.compile('(VODAFON|Cosmote|WIND|FORTHNET)\S*' ,re.IGNORECASE)
p57 = re.compile('(DIDAKTRA|ΔΙΔΑΚΤΡΑ)\S*' ,re.IGNORECASE)
p59 = re.compile('(ΝΟΜΟΣ 3869|ΝΟΜΟΣ3869|Ν.3869)\S*' ,re.IGNORECASE)
p60 = re.compile('(Ασφαλεια αυτο/του|ασφαλεια αυτοκινητου|ασφαλειων αυτ/των|ασφαλεια αυτοκινητο|Ασφαλιστηριο αυτοκινητου|Ασφαλιστρα αυτοκινητου|ασφαλιση αυτοκινητου)\S*' ,re.IGNORECASE)
p61 = re.compile('(Ασφαλεια σπιτιου|asfaleia spitiou|ΑΣΦΆΛΕΙΑ ΣΠΙΤΙΟΎ|asfalia spitiou)\S*' ,re.IGNORECASE)
p62 = re.compile('(enoik|noiki|rent|νοικι)\S*' ,re.IGNORECASE)
p63 = re.compile('(payment|payments|plhpomh|pliromes|pliromi|pliromis|πληρωμες|πληρωμη)\S*' ,re.IGNORECASE)
p64 = re.compile('(συμβ|συμβολαιο|συμβολαιου|symv)\S*' ,re.IGNORECASE)
p65 = re.compile('(επιδομα|επιδοματος|epidoma|epidomatos)\S*' ,re.IGNORECASE)
p66 = re.compile('(κρατηση|κρατησης|kratisi|kratisis|booking)\S*' ,re.IGNORECASE)
p67 = re.compile('(diakopes|διακοπες|κρατηση δωματιου|kratisi dwmatiou|kratisi domatiou|κρατηση ξενοδοχειου|κρατηση δικλινου|κρατηση τρικλινου|κρατηση καταλυματος|κρατηση δωματιων|δικλινο|τρικλινο|diklino|triklino|δικλινου|τρικλινου|diklinou|triklinou)\S*' ,re.IGNORECASE)
p68 = re.compile('(petrelaio|πετρελαιο|πετρελαιου|petreleo|petrelaiou)\S*' ,re.IGNORECASE)
p69 = re.compile('(kausima|kafsima|καυσιμα)\S*' ,re.IGNORECASE)
p71 = re.compile('(αγορα|αγορας|αγορες|agora|agoras)\S*' ,re.IGNORECASE)
p75 = re.compile('(ANGLIKA|agglika|gallika|αγγλικα|γαλλικα|ΦΡΟΝΤΙΣΤΗΡΙΟ|frontistirio|frontistiriou|frontisthrio|φροντιστηριου)\S*' ,re.IGNORECASE)
 

#timologia
p35 = re.compile('\s?(tim|τιμ)\s?\.?\d{1,30}\s?',re.IGNORECASE)
p72 = re.compile('\s?(τδα|tda)\s?\.?\d{1,30}\s?',re.IGNORECASE)
p46 = re.compile('\s?(inv)\s?\.?\d{1,30}\s?',re.IGNORECASE)
#μηνες για εξαιρεση από πινακιδες
p73 = re.compile('\s?(ιαν|φεβ|μαρ|απρ|μαι|ιου|αυγ|σεπ|οκτ|νοε|δεκ)\s?\.?\d{1,30}\s?',re.IGNORECASE)
p74 = re.compile('\s?(jan|feb|mar|apr|may|jun|jul|aug|sep|okt|nov|dec)\s?\.?\d{1,30}\s?',re.IGNORECASE)
#afm
p36 = re.compile('\s?(αφμ|afm)\s?\.?\d{1,9}\s?',re.IGNORECASE)
p37 = re.compile('(ΑΦΜ|AFM)\S*' ,re.IGNORECASE)
#transfers
p39 = re.compile('\s?(POO)\s?\.?\s?',re.IGNORECASE)
p40 = re.compile('\s?(POI)\s?\.?\s?',re.IGNORECASE)
#characters
p58 = re.compile('\s?[\W|_]{1}\s?',re.IGNORECASE)


<b>Functions definitions</b>

In [39]:
def loadStopWords(ws):
    #A dataset containing the Greek stop words has been created
    #the function loads this dataset as a dataframe
    dataset = Dataset.get_by_name(ws, name='stopWords_gr')
    sw = set(dataset.to_pandas_dataframe())
    return sw

In [40]:
def replaceTerm(text):
    #This function uses the above defined regular expressions to replace text
    #The order of the rules is importand
    #Compinations of two or more words, are concatenated, in order to be considered as a single token
    
    #text = str(text)
    text = p58.sub(' ',text)
    
    text = p5.sub(' λογαριασμος ',text)
    text = p4.sub(' τηλεφωνο ',text)
    text = p6.sub(' δενθελειδενενδιαφερεται ',text)
    text = p10.sub(' δενθελειδενενδιαφερεται ',text)
    text = p7.sub(' δενεχειδενμπορει ',text)
    text = p8.sub(' δενειναιδιαθεσιμος ',text)
    text = p9.sub(' ανεφικτη ',text)
    text = text.replace('-banking','banking')
    text = text.replace('v banking','vbanking')
    text = text.replace('e banking','ebanking')
    text = text.replace('follow up','followup')
    text = text.replace('fup','followup')
    text = text.replace('f/up','followup')
    text = text.replace('πυρ/ριο','πυρασφαλιστηριο')
    text = text.replace('safe drive','safedrive')
    text = text.replace('safe pocket','safepocket')
    text = text.replace('alphabank','alpha')
    text = text.replace('sweet home smart','sweethomesmart')
    text = text.replace('sweet home','sweethome')
    text = text.replace('eξασφαλιζω','εξασφαλιζω')
    text = text.replace('credit card','creditcard')
    text = text.replace('debit card','debitcard')
    text = text.replace('life cycle','lifecycle')
    text = text.replace('π/κ','πκ')
    text = text.replace('td','πκ')
    text = text.replace('α/κ','ακ')
    text = text.replace('δ/α','δεναπαντα ')
    text = text.replace('εκτος αττικης','εκτοςαττικης ')
    text = text.replace('paf payments','συναλλαγηpaf')
    text = text.replace('paf online payments','συναλλαγηpaf')
    text = text.replace('κτθ aps', 'καταθεσηaps')
    text = text.replace('καταθεση μετρ. απο aps', 'καταθεσηaps')
    text = text.replace('μτφ μεσω web','συστημικόμτφebanking')
    text = text.replace('visa gold', 'visagold')

    
    
    #τδ
    text = p1.sub(' δεναπαντα ',text)
    text = p2.sub(' δεναπαντα ',text)
    text = p3.sub(' δεντονβρηκα ',text)
    text = p37.sub(' αριθμοςφορολογικουμητρωου ',text)
    text = p36.sub(' αριθμοςφορολογικουμητρωου ',text)
    text = p35.sub(' τιμολογια ',text)
    text = p46.sub(' τιμολογια ',text)
    text = p72.sub(' τιμολογια ',text)
    text = p17.sub(' τιμολογια ',text)
    text = p73.sub(' μηνες ',text)
    text = p74.sub(' μηνες ',text)
    text = p11.sub(' αριθμοςκυκλοφοριας ', text)
    text = p12.sub(' ενοικιο ',text)
    text = p13.sub(' ενοικιο ',text)
    text = p14.sub(' γιατροι ',text)
    text = p15.sub(' εξοφληση ',text)
   
    
    text = p18.sub(' ΝΕΡΟ ',text)
    text = p38.sub(' ΡΕΥΜΑ ',text)
    text = p19.sub(' μισθοδοσιες ', text)
    text = p20.sub(' εφοριαακινητων ', text)
    text = p41.sub(' ΕΝΦΙΑ ', text)
    text = p21.sub(' φοροςεισοδηματος ', text)
    text = p42.sub(' ΕΦΚΑ ', text)
    text = p43.sub(' ΦΠΑ ', text)
    text = p44.sub(' ΙΚΑ ', text)
    text = p45.sub(' ΤΕΒΕ ', text)
    text = p22.sub(' κοινοχρηστα ', text)
    text = p23.sub(' συναλλαγεςγαμου ', text)
    text = p24.sub(' συναλλαγεςγεννησηςβαφτισης ', text)
    text = p25.sub(' στεγαστικα ', text)
    text = p26.sub(' μεταφορες ', text)
    text = p27.sub(' παραγγελιες ', text)
    text = p28.sub(' οφειλες ', text)
    text = p29.sub(' καταθεσεις ', text)
    text = p30.sub(' προκαταβολες ', text)
    text = p31.sub(' διατροφη ', text)
    text = p32.sub(' carloan ', text)
    text = p33.sub(' δανειο ', text)
    text = p34.sub(' πιστωτικη ', text)
    text = p39.sub(' εξερχομενοεμβασμα ',text)
    text = p40.sub(' εισερχομενοεμβασμα ',text)
    text = p47.sub(' δωρεα ',text)
    text = p48.sub(' δοσεις ',text)
    text = p49.sub(' τραπεζεςανταγωνισμου ',text)
    text = p70.sub(' τραπεζεςανταγωνισμου ',text)
    text = p50.sub(' ιδιωτικάσχολεια ',text)
    text = p51.sub(' ΟΑΕΔ ',text)
    text = p52.sub(' τεληκυκλοφοριας ',text)
    text = p53.sub(' επιταγες ',text)
    text = p54.sub(' μερισματα ',text)
    text = p55.sub(' κολλεγια ',text)
    text = p56.sub(' τηλεπικοινωνιες ',text)
    text = p57.sub(' διδακτρα ',text)
    text = p59.sub(' ΝΟΜΟΣ3869 ',text)
    text = p60.sub(' ασφαλειααυτοκινητου ',text)
    text = p61.sub(' ασφαλειασπιτιου ',text)
    text = p16.sub(' ασφαλειες ',text)
    text = p62.sub(' ενοικιο ',text)
    text = p63.sub(' πληρωμες ',text)
    text = p64.sub(' συμβολαια ',text)
    text = p65.sub(' επιδομα ',text)
    text = p67.sub(' διακοπες ',text)
    text = p66.sub(' κρατησεις ',text)
    text = p68.sub(' πετρελαιο ',text)
    text = p69.sub(' καυσιμα ',text)
    text = p77.sub(' αγορααυτοκινητου ',text)
    text = p71.sub(' αγορες ',text)
    text = p75.sub(' φροντιστηρια ',text)
    text = p76.sub(' αυτοκινητο ',text)
    
    
    return text

In [41]:
def remove_ton(text):
    #removes punctuation, αφαιρεί τους τόνους
    diction = {'ά':'α','έ':'ε','ή':'η','ί':'ι','ό':'ο','ώ':'ω','ύ':'υ'}
    for key in diction.keys():
        text = text.replace(key, diction[key])
    return text   

In [42]:
def clean_text(text,sw):
    #This function performs text cleansing and returns the clean and lemmatized version of the original text
    #conver to lower text 
    text = str(text).lower()
    
    # remove puncutation
    #create a list from each text
    text = [word.strip(string.punctuation) for word in text.split(" ")]

    # αφαιρούνται οι τόνοι
    text = [remove_ton(x) for x in text]
    
    # remove stop words
    text = [x for x in text if x not in sw]
    
    #remove quotes
    text = [x.replace('quot;','').replace('&quot','') for x in text if x not in {'quot','amp'}]
    
    # remove words that contain numbers
    #text = [word for word in text if not any(c.isdigit() for c in word)] #addition to return even empty tokens
    
    # remove empty tokens
    text = [t for t in text if len(t) > 2] #addition to return even empty tokens
    
    # remove amp & quot
    text = [x for x in text if x not in ['quot','amp']]
    
    # remove words with only one letter
    #recreate text from list
    text = " ".join([t for t in text if len(t) > 2]) #addition to return even empty tokens
    
     #replacements either by rules or regular expressions
    text = replaceTerm(text)
    
    # lemmatize text
    text = " ".join([t.lemma_ for t in nlp(text, disable=['tagger', 'parser', 'ner','tok2vec', 'morphologizer', 'parser', 'senter', 'attribute_ruler',  'ner'])])
    return(text)

In [43]:
def load_correctDict(ws):
    #it creates a dictionary out of a dataset that containes pairs of (original term, corrected term)    
    dataset = Dataset.get_by_name(ws, name='correct_Tokens')    
    corDict = dict(dataset.to_pandas_dataframe().to_dict("split")['data'])
    return corDict

In [44]:
def correct(x,corDict):
    #uses the dictionary to correct the terms
    if x in corDict.keys():
        y = corDict[x]
    else:
        y = x
    return y    

In [45]:
def get_ngrams(idf,mindf,minngram,maxngram):
    #this function returns the bi-grams and tri-grams
    tfidf = TfidfVectorizer(min_df = mindf,ngram_range = (minngram,maxngram))
    tfidf_result = tfidf.fit_transform(idf['tokenized']).toarray()
    tfidf_df = pd.DataFrame(tfidf_result, columns = tfidf.get_feature_names())
    tfidf_df.columns = [str(x) for x in tfidf_df.columns]
    df_i = pd.concat([df[['CON_ROW_ID']],tfidf_df],axis=1).melt(id_vars=['CON_ROW_ID'],value_vars = tfidf_df.columns).dropna()
    df_i = df_i[df_i['value']>0]
    return df_i

In [46]:
def cleanComments(df,sw):
    #applies the clean text function to all texts contained in the dataset
    df = df[['CON_ROW_ID','CON_COMMENTS']]
    df['tokenized'] = df['CON_COMMENTS'].apply(clean_text,args = (sw))
    df = df.fillna('N/A')
    df['variable'] = df['tokenized'].str.split()
    return df

In [47]:
def getTokens(df,sw):
    #The variable columns is a list. The explode method "unpivots this list"
    df = cleanComments(df,sw)
    df_f = df.explode('variable')[['CON_ROW_ID','variable']]
    return df_f

In [48]:
def getTokencount(df_f,minCount):
    #calculate the number of occurances (counts) of each token
    #tokens with count less than mincount are set to blank
    tokenCount = df_f['variable'].value_counts().to_dict()
    
    df_f['value'] = df_f['variable'].map(tokenCount)
   
    df_f.loc[(df_f['value']<minCount), 'variable'] = ' ' #addition to return even empty tokens
    
    return df_f

In [49]:
def performNLP(workspace,minCount,mindf,minngram,maxngram,deleteEmptyTokens,df):
    sw = loadStopWords(workspace)
    
    df = cleanComments(df,sw)
    
    df_f = getTokens(df,sw)

    df_f.count()
    
    df_f = df_f.fillna(' ')
    
    df_f = getTokencount(df_f,minCount)
    
    #try:
    #    df_f = df_f.append(get_ngrams(df,mindf,minngram,maxngram ))
    #except:
    #    print('no bigramms or trigramms were added')
    
    #corDict = load_correctDict(workspace)     
    
    #df_f['token'] = df_f['variable'].apply(lambda x : correct(x,corDict))
    
    #df_f.loc[(df_f['variable'].str.len() <2), 'variable'] = ' ' #single character tokens are set to blank
    
    df_f['token'] = df_f['variable']
    
    df_f['length'] = df_f['token'].str.len() #tokens length
    
    df_f.loc[(df_f['length']<3), ['token']] = ' ' #till two-character tokens are set to blank
    
    df_f = df_f.sort_values(['CON_ROW_ID','token'])
    
    if deleteEmptyTokens:
        df_f = df_f[df_f['token'] != ' ']
    
    df_f = df_f[['CON_ROW_ID','token']].drop_duplicates()
    
    return df_f

In [50]:
def loadTexts(workspace,datasetName):
    #loads the texts to be analyzed
    dataset = Dataset.get_by_name(workspace, name=datasetName)
    df = dataset.to_pandas_dataframe()
    df= df[['CON_ROW_ID','CON_COMMENTS']]
    return df   

In [51]:
def exportResults(workspace,datastore,fileName,df_f):
    df_f.to_csv(fileName+'.txt',sep ='|',line_terminator='\r\n',index = False, encoding = 'UTF-8')
    fil = [os.getcwd()+'/'+ fileName+'.txt']
    #datastore.upload_files(fil, target_path='UI/NLP', overwrite=True, show_progress=True)
    
    

<b>The commended-out code is for debuging purposes</b>

In [52]:
#txt = 'H eurobank είναι καλύτερη τράπεζα στον κόσμο'
#com = {'CON_ROW_ID':[1],'CON_COMMENTS':[txt]}
#df = pd.DataFrame(com)

In [53]:
df = loadTexts(workspace,datasetName)

In [54]:
#df = df.head(100000)

In [55]:
df.head(1000)

,CON_ROW_ID,CON_COMMENTS
0,29059570,POO SPYROS FOTEINOPOULOS &
1,29059571,POO KALKASINA STYLIANI MICHAIL&
2,29059572,Αμαλία Τζατζουλη Φεβρουάριο 21&
3,29059573,COM POO ADAMAKOS EPE SE AFOI S&
4,29059574,POI STATHOGLOY GEORGIOS &
...,...,...
995,29004196,POI 0227da7a-8612-4e8d-aaae-80&
996,29004197,POI kotronakis saridakis oe in&
997,29004198,Θ ΠΑΓΩΝΗΣ Γ ΚΟΧΡΩΝΑ ΕΛΤΑ ΚΩΔΙΚ&
998,29004199,ΕΙΣΠΡΑΞΕΙΣ ΤΗΝ 25/01/21 ΔΕ ΦΕΝ&


In [56]:
df_f = performNLP(workspace, minCount, mindf, minngram, maxngram, deleteEmptyTokens, df)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [57]:
#df_f.head(1000)

In [58]:
exportResults(workspace,datastore,fileName,df_f)

#run.complete()UT

In [59]:
#pd.merge(df_f,df, how="inner",on=['CON_ROW_ID']).head(1000000).to_excel('NLP{0}.xlsx'.format(fileName))

In [60]:
#df_f['token'].value_counts()

In [61]:
#df[df['CON_ROW_ID']==6530134]

In [62]:
#df_f[df_f['token']=='αριθμοςκυκλοφοριας']
